In [ ]:
from ultralytics import YOLO

model = YOLO('models/ball_best.pt')
model.export(format='onnx', dynamic=True)

In [ ]:
from ultralytics import YOLO
model = YOLO('models/ball_best.onnx', task='detect')
results = model.predict(source='1765199807.mp4', conf=0.3, save=True, batch = 64)

In [2]:
import cv2
import json

VIDEO_PATH = "1765199807.mp4"
OUTPUT_JSON = "data.json"

point_names = [
    "top_left",
    "top_center",
    "top_right",
    "center_right",
    "bottom_right",
    "bottom_center",
    "bottom_left",
    "center_left"
]

points = {}
click_count = 0

def mouse_callback(event, x, y, flags, param):
    global click_count, frame

    if event == cv2.EVENT_LBUTTONDOWN:
        if click_count < len(point_names):
            name = point_names[click_count]
            points[name] = {"x": int(x), "y": int(y)}

            # vẽ điểm
            cv2.circle(frame, (x, y), 6, (0, 0, 255), -1)
            cv2.putText(
                frame,
                name,
                (x + 10, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2
            )

            click_count += 1
            print(f"[{click_count}/8] {name}: ({x}, {y})")

            if click_count == len(point_names):
                save_json()
                print("✅ Đã lưu file JSON")

def save_json():
    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(points, f, indent=4, ensure_ascii=False)

# ---- Đọc video ----
cap = cv2.VideoCapture(VIDEO_PATH)
ret, frame = cap.read()
cap.release()

if not ret:
    raise RuntimeError("❌ Không đọc được video")

cv2.namedWindow("Select Points")
cv2.setMouseCallback("Select Points", mouse_callback)

print("📌 Click theo thứ tự:")
for i, name in enumerate(point_names):
    print(f"{i+1}. {name}")

# ---- Loop hiển thị ----
while True:
    cv2.imshow("Select Points", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == 27:  # ESC
        break

cv2.destroyAllWindows()


📌 Click theo thứ tự:
1. top_left
2. top_center
3. top_right
4. center_right
5. bottom_right
6. bottom_center
7. bottom_left
8. center_left
[1/8] top_left: (171, 325)
[2/8] top_center: (271, 221)
[3/8] top_right: (363, 139)
[4/8] center_right: (559, 128)
[5/8] bottom_right: (981, 154)
[6/8] bottom_center: (887, 343)
[7/8] bottom_left: (715, 643)
[8/8] center_left: (294, 406)
✅ Đã lưu file JSON
